In [1]:
import os
from src.exception import CustomException
import sys
print(os.getcwd())

/home/tanush-reddy/workspace/Wine-Grade-Estimator/research


In [2]:
os.chdir("../")

In [3]:
print(os.getcwd())

/home/tanush-reddy/workspace/Wine-Grade-Estimator


In [19]:
from dataclasses import dataclass
from pathlib import Path

@dataclass
class DataIngestionConfig:
    root_dir: Path
    source_url: str
    local_data_file: Path
    unzip_dir: Path

In [20]:
from src.constants import CONFIG_FILE_PATH, PARAMS_FILE_PATH, SCHEMA_FILE_PATH
from src.utils.common import read_yaml, create_directories


In [21]:
class ConfigurationManager:
    def __init__(self, config_file_path=CONFIG_FILE_PATH, params_file_path=PARAMS_FILE_PATH, schema_file_path=SCHEMA_FILE_PATH):
        # ✅ Save parameters as instance variables
        self.config_file_path = config_file_path
        self.params_file_path = params_file_path
        self.schema_file_path = schema_file_path

        # ✅ Now you can use them
        self.config = read_yaml(self.config_file_path)
        self.params = read_yaml(self.params_file_path)
        self.schema = read_yaml(self.schema_file_path)

        create_directories([self.config.artifacts_root])

        
    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config.data_ingestion
        create_directories([config.root_dir, config.unzip_dir])
        
        data_ingestion_config = DataIngestionConfig(
            root_dir=Path(config.root_dir),
            source_url=config.source_url,
            local_data_file=Path(config.local_data_file),
            unzip_dir=Path(config.unzip_dir)
        )
        
        return data_ingestion_config
    
    
    

In [22]:
import os
import urllib.request as request
import zipfile
from src.utils.common import get_size
from src import logger

In [27]:
class DataIngestion:
    def __init__(self, config: DataIngestionConfig):
        self.config = config

    def download_data(self):
        if not os.path.exists(self.config.local_data_file):
            logger.info(f"Downloading data from {self.config.source_url} to {self.config.local_data_file}")
            filename, headers=request.urlretrieve(url=self.config.source_url,filename= self.config.local_data_file)
            # logger.info(f"Downloaded {get_size(self.config.local_data_file)} bytes")

    def unzip_data(self):
        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path, exist_ok=True)
        with zipfile.ZipFile(self.config.local_data_file, 'r') as zip_ref:
            zip_ref.extractall(unzip_path)
            
    


In [28]:
from box import ConfigBox

data = ConfigBox({"name": "Tanush", "age": 21})

print(data.name)  # instead of data["name"]
print(data.age)   # instead of data["age"]


Tanush
21


In [ ]:
try:
    config=ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config=data_ingestion_config)
    data_ingestion.download_data() 
    data_ingestion.unzip_data()
except Exception as e:
    raise CustomException(e, sys) from e

[2025-07-28 19:21:20,222] 28 WineGradeEstimator - INFO - YAML file config/config.yaml read successfully.
[2025-07-28 19:21:20,224] 28 WineGradeEstimator - INFO - YAML file params.yaml read successfully.
[2025-07-28 19:21:20,225] 28 WineGradeEstimator - INFO - YAML file schema.yaml read successfully.
[2025-07-28 19:21:20,226] 46 WineGradeEstimator - INFO - Directory created: artifacts
[2025-07-28 19:21:20,227] 46 WineGradeEstimator - INFO - Directory created: artifacts/data_ingestion
[2025-07-28 19:21:20,228] 46 WineGradeEstimator - INFO - Directory created: artifacts/data_ingestion


In [ ]:
import os